### Assignment-01-附加题-极具挑战性的题目，基础一般的同学请把前边的做好就非常非常好了，这个是一个趣味题 😝

## 基于模式匹配的对话机器人实现

### Pattern Match

机器能否实现对话，这个长久以来是衡量机器人是否具有智能的一个重要标志。 Alan Turing早在其文中就提出过一个测试机器智能程度的方法，该方法主要是考察人类是否能够通过对话内容区分对方是机器人还是真正的人类，如果人类无法区分，我们就称之为具有”智能“。而这个测试，后来被大家叫做”图灵测试“，之后也被翻拍成了一步著名电影，叫做《模拟游戏》。 



既然图灵当年以此作为机器是否具备智能的标志，这项任务肯定是复杂的。自从 1960s 开始，诸多科学家就希望从各个方面来解决这个问题，直到如今，都只能解决一部分问题。 目前对话机器人的建立方法有很多，今天的作业中，我们为大家提供一共快速的基于模板的对话机器人配置方式。

此次作业首先希望大家能够读懂这段程序的代码，其次，在此基于我们提供的代码，**能够把它改造成汉语版本，实现对话效果。**

```
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)
```

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成  "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

In [63]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [64]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]): return True
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

### 例如

In [65]:
pat_match('I want ?X'.split(), "I want holiday".split())

True

In [66]:
pat_match('I have dreamed a ?X'.split(), "I dreamed about dog".split())

False

In [67]:
pat_match('I dreamed about ?X'.split(), "I dreamed about dog".split())

True

### 获得匹配的变量

以上的函数能够判断两个 pattern 是不是相符，但是我们更加希望的是获得每个variable对应的是什么值。

我们对程序做如下修改:

In [68]:
def pat_match(pattern, saying):
    if is_variable(pattern[0]):
        return pattern[0], saying[0]
    else:
        if pattern[0] != saying[0]: return False
        else:
            return pat_match(pattern[1:], saying[1:])

In [69]:
pattern = 'I want ?X'.split()
saying = "I want holiday".split()

In [70]:
pat_match(pattern, saying)

('?X', 'holiday')

In [71]:
pat_match("?X equals ?X".split(), "2+2 equals 2+2".split())

('?X', '2+2')

但是，如果我们的 Pattern 中具备两个变量，那么以上程序就不能解决了，我们可以对程序做如下修改: 


In [72]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

于是，我们可以获得： 

In [73]:
pat_match("fa ?X greater than ?Y".split(), "seker 3 greaterd thand 2".split())

[]

如果我们知道了每个变量对应的是什么，那么我们就可以很方便的使用我们定义好的模板进行替换：

为了方便接下来的替换工作，我们新建立两个函数，一个是把我们解析出来的结果变成一个 dictionary，一个是依据这个 dictionary 依照我们的定义的方式进行替换。

In [74]:
def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

In [75]:
def substitute(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + substitute(rule[1:], parsed_rules)

In [76]:
got_patterns = pat_match("I want3 ?X".split(), "I want iPhone".split())

In [77]:
got_patterns

[]

In [78]:
substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', '?X']

为了将以上输出变成一句话，也很简单，我们使用 Python 的 join 方法即可： 

In [79]:
john_pat = pat_match('?P needs ?X'.split(), "John needs resting".split())

In [81]:
' '.join(substitute("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a ?X'

In [82]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())

In [85]:
substitute("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'vacation', '?']

In [86]:
' '.join(substitute("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

那么如果我们现在定义一些patterns，就可以实现基于模板的对话生成了:

In [87]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [91]:
import random
def get_response(saying):
    result = []
    for k, v in defined_patterns.items():
        t = pat_match(k.split(), saying.split())
        if t:
            result = t, v
    dictionary = pat_to_dict(result[0])
    answer = random.choice(result[1])
    return ' '.join(substitute(answer.split(), dictionary))

In [92]:
get_response('I need iPhone')

'Why do you need iPhone ?'

In [93]:
get_response('My mother told me something')

'How do you think about your mother ?'

### Segment Match

我们上边的这种形式，能够进行一些初级的对话了，但是我们的模式逐字逐句匹配的， "I need iPhone" 和 "I need ?X" 可以匹配，但是"I need an iPhone" 和 "I need ?X" 就不匹配了，那怎么办？ 

为了解决这个问题，我们可以新建一个变量类型 "?\*X", 这种类型多了一个星号(\*),表示匹配多个

首先，和前文类似，我们需要定义一个判断是不是匹配多个的variable

In [94]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [95]:
is_pattern_segment('?*P')

True

In [96]:
from collections import defaultdict

然后我们把之前的 ```pat_match```程序改写成如下， 主要是增加了 ``` is_pattern_segment ```的部分. 

In [175]:
fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        t = pat_match_with_seg(pattern[1:], saying[index:])
        return [match] + t if match[0][1] else t
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail

这段程序里比较重要的一个新函数是 ```segment_match```，这个函数输入是一个以 ```segment_pattern```开头的模式，尽最大可能进行，匹配到这个*变长*的变量对应的部分。

In [174]:
def segment_match(pattern, saying):
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    if not rest: return (seg_pat, saying), len(saying)    
    
    for i, token in enumerate(saying):
        # Modify condition like pat_match_with_seg('?*X hello'.split(), 'hi hello you'.split()) raise an index error.
        try:
            if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
                return (seg_pat, saying[:i]), i
        except IndexError:
            return (seg_pat, saying[:i]), i
    return (seg_pat, saying), len(saying)

def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]):
        return True
    if rest[0] != saying[0]:
        return False
    return is_match(rest[1:], saying[1:])

In [99]:
segment_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

现在，我们就可以做到以下的匹配模式了: 

In [100]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

如果我们继续定义一些模板，我们进行匹配，就能够进行更加复杂的问题了: 

In [101]:
response_pair = {
    'I need ?X': [
        "Why do you neeed ?X"
    ],
    "I dont like my ?X": ["What bad things did ?X do for you?"]
}

In [102]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [105]:
substitute("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

 我们会发现，pat_to_dict在这个场景下会有有一点小问题，没关系，修正一些: 

In [106]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [109]:
substitute("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

如果我们定义这样的一个模板:

In [110]:
("?*X hello ?*Y", "Hi, how do you do")

('?*X hello ?*Y', 'Hi, how do you do')

In [112]:
substitute("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

### 现在是你的时间了

In [113]:
#我们给大家一些例子: 
    
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

### 问题1

编写一个程序, ```get_response(saying, response_rules)```输入是一个字符串 + 我们定义的 rules，例如上边我们所写的 pattern， 输出是一个回答。

In [114]:
def get_response_star(saying, response_rules=rule_responses):
    result = []
    for question, answer in response_rules.items():
        temp = pat_match_with_seg(question.split(), saying.split())
        if len(temp) > 1 and None not in temp:
            result = substitute(random.choice(answer).split(), pat_to_dict(temp))
    if not result:
        return random.choice(response_rules['?*x'])
    return ' '.join(result)

In [115]:
pat_match_with_seg('?*X hello'.split(), 'hi hello you'.split())

[('?X', ['hi'])]

In [116]:
pat_match_with_seg('?*X hello ok'.split(), 'hi hello you'.split())

[('?X', ['hi', 'hello', 'you'])]

In [117]:
pat_match_with_seg('?*X'.split(), 'hi hello you'.split())

[('?X', ['hi', 'hello', 'you'])]

In [118]:
pat_match_with_seg('what ?*X fdakljf'.split(), 'hi you there'.split())

[True, None]

In [119]:
get_response_star('cute  world')

'很有趣'

In [120]:
temp = pat_match_with_seg('?*X hello ?*y'.split(), 'cute hello world'.split())
temp

[('?X', ['cute']), ('?y', ['world'])]

### 问题2

改写以上程序，将程序变成能够支持中文输入的模式。
*提示*: 你可以需用用到 jieba 分词

In [265]:
import jieba
def new_split(string):
    """Compatible with English mode."""
    result = []
    temp = list(jieba.cut(string))
    for i, each in enumerate(temp):
        if each == '?':
            if len(temp) == i + 1:
                result.append(each)
            elif temp[i+1] == '*':
                result.append(temp[i] + temp[i + 1] + temp[i + 2])
            elif temp[i+1].isalpha():
                result.append(temp[i] + temp[i + 1])
        elif len(each) == 1 and (each == '*' or each.isupper() or each.islower()):
            # ignore '*' and alphabet following the '?'
            continue
        elif each != ' ':
            result.append(each)
    return result

def get_response_star_new(saying, response_rules=rule_responses):
    result = []
    last = []
    for question, answer in response_rules.items():
        temp = pat_match_with_seg(new_split(question), new_split(saying))
        # pop the ('?x", '') tuple in the pattern match list temp.
        for k, v in temp:
            if not v:
                temp.pop(temp.index((k, v)))
        if len(temp) > 1 and None not in temp:
            result = substitute(new_split(random.choice(answer)), pat_to_dict(temp))
    if not result:
        return random.choice(response_rules['?*x'])
    for each in result:
        last.append(''.join(each.split()))
        # consider use language model with result to make answer sentence more intellgent.
    return ''.join(last)

In [133]:
new_split('?*x我很难过，因为?*y')

['?*x', '我', '很', '难过', '，', '因为', '?*y']

In [135]:
new_split('我很难过，?x因为?y')

['我', '很', '难过', '，', '?x', '因为', '?y']

In [136]:
pat_match_with_seg(new_split('?*x我?*y你?*Z'), new_split('我讨厌他'))

[('?x', []), ('?y', ['讨厌', '他'])]

In [137]:
get_response_star_new('我讨厌他')

'你不想要他吗？'

In [138]:
get_response_star_new('你知道么我昨晚梦见他')

'其实很有可能我们互相昨晚梦见他'

In [139]:
segment_match(new_split('?*x我很难过，因为?*y'), new_split('朋友我很难过，因为喝酒了'))

(('?x', ['朋友']), 1)

In [140]:
segment_match('?*X hello world ?*Y'.split(), 'welcom hello world new boy'.split())

(('?X', ['welcom']), 1)

In [141]:
pat_match_with_seg('?*X hello world ?*Y'.split(), 'welcom hello world new boy'.split())

[('?X', ['welcom']), ('?Y', ['new', 'boy'])]

In [177]:
get_response_star_new('我讨厌他')

'他有什么不好呢？'

In [178]:
get_response_star_new('他可能很厉害')

'你看起来不太确定'

In [143]:
get_response_star_new('你一直这么干')

'你能想到一些其他情况吗?'

In [144]:
get_response_star_new('小草就像大树')

'你觉得小草和大树有什么相似性？'

In [145]:
get_response_star_new('他妈妈很贤惠')

'你家里除了很贤惠还有谁?'

In [146]:
get_response_star_new('我不要和你一样')

'你有一点负能量'

In [147]:
get_response_star_new('他总是这么聪明')

'你能想到一些其他情况吗?'

In [148]:
get_response_star_new('难道所有人都这样么？')

'我确定不是人人都是'

In [159]:
get_response_star_new('我和他一样帅')

'帅会对你有什么影响呢?'

In [158]:
get_response_star_new('我妈妈对我很好')

'你家里除了对我很好还有谁?'

In [162]:
get_response_star_new('为什么我不能飞')

'其实很有可能我们互相不能飞'

In [164]:
get_response_star_new('他说我不能这样')

'其实很有可能我们互相不能这样'

In [165]:
get_response_star_new('你说我是神吗')

'其实很有可能我们互相是神吗'

* [x] 上面三个例子应该匹配：(?*x和?*y一样?*z)；(?*x妈妈?*y)；以及(?*x我不能?*y)，但是都被(?*x我?*y你?*z)匹配，需改进回答函数get_response_star_new()
        先注释掉这条模板= =
* [x] 更新函数`get_response_star_new_v2`，以上测试不会出现输入输出多对一的情况，见末尾。

# **待调整以适应更多模型匹配**

### 问题3

多设计一些模式，让这个程序变得更好玩，多和大家交流，看看大家有什么好玩的模式

### 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？     
2. 什么是数据驱动？数据驱动在这个程序里如何体现？
3. 数据驱动与 AI 的关系是什么？ 

1. 优点：pass。缺点挺明显的：需要严格匹配模板格式；中文分词会导致同时匹配多个定义模糊的模式，得到结果出现答非所问的情况。
    * [x] 原模式匹配函数：`pat_match_with_seg('?*X hello'.split(), 'hi hello you'.split())`该函数中的匹配参数与语句参数如左所示，将会出错，需改进子函数 `segment_match`；
    * [x] 空字符也算匹配，导致失配，已改进。
    * [ ] 提高匹配精度，让某句话只能匹配词库中的一条，否则返回['?*X']的值，避免出现如上多个例子被一个模式匹配的情况。该策略是否得当待后期课程解惑。
    * [ ] `new_split`函数对于带英文的分词需要改善，如‘?*xAI?*y’会被分成['?*xAI', 'xAI', '?*y']，尝试不用jieba实现，自己实现中文单个切割，英文单个切割
2. 模型的正确率取决于模式定义，定义的准确性需要数据支撑。
3. 基于训练模型的AI应用，数据是基础。

一些参考 pattern

In [335]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那意味着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?y', '如果你能?y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}


In [319]:
v = ['你觉得我是神吗'] * len(rule_responses.keys())
v = list(map(new_split, v))
v

[['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '我', '是', '神', '吗'],
 ['你', '觉得', '

In [321]:
v1 = list(map(new_split, list(rule_responses.keys())))
v1

[['?*x', 'hello', '?*y'],
 ['?*x', 'want', '?*y'],
 ['?*x', 'if', '?*y'],
 ['?*x', 'no', '?*y'],
 ['?*x', 'was', '?*y'],
 ['?*x', 'feel', '?*y'],
 ['?*x', '你好', '?*y'],
 ['?*x', '我', '想', '?*y'],
 ['?*x', '我', '想要', '?*y'],
 ['?*x', '喜欢', '?*y'],
 ['?*x', '讨厌', '?*y'],
 ['?*xAI', 'xAI', '?*y'],
 ['?*x', '机器人', '?*y'],
 ['?*x', '对不起', '?*y'],
 ['?*x', '我', '记得', '?*y'],
 ['?*x', '如果', '?*y'],
 ['?*x', '我', '?*z', '梦见', '?*y'],
 ['?*x', '妈妈', '?*y'],
 ['?*x', '爸爸', '?*y'],
 ['?*x', '我', '愿意', '?*y'],
 ['?*x', '我', '很', '难过', '，', '因为', '?*y'],
 ['?*x', '难过', '?*y'],
 ['?*x', '就', '像', '?*y'],
 ['?*x', '和', '?*y', '都', '?*z'],
 ['?*x', '和', '?*y', '一样', '?*z'],
 ['?*x', '我', '是', '?*y'],
 ['?*x', '我', '是', '?*y', '吗'],
 ['?*x', '你', '是', '?*y', '吗'],
 ['?*x', '你', '是', '?*y'],
 ['?*x', '因为', '?*y'],
 ['?*x', '我', '不能', '?*y'],
 ['?*x', '我', '觉得', '?*y'],
 ['?*x', '我', '?*y', '你', '?*z'],
 ['?*x', '你', '为什么', '不', '?*y'],
 ['?*x', '好', '的', '?*y'],
 ['?*x', '嗯', '嗯', '?*y'],
 ['?*x', '不嘛',

In [323]:
temp_list = list(map(pat_match_with_seg, v1, v))
temp_list

[[('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?xAI', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得']), ('?z', ['是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我', '是', '神', '吗'])],
 [('?x', ['你', '觉得', '我',

In [314]:
d = list(map(set, list(map(new_split, list(rule_responses.keys())))))
d

[{'?*x', '?*y', 'hello'},
 {'?*x', '?*y', 'want'},
 {'?*x', '?*y', 'if'},
 {'?*x', '?*y', 'no'},
 {'?*x', '?*y', 'was'},
 {'?*x', '?*y', 'feel'},
 {'?*x', '?*y', '你好'},
 {'?*x', '?*y', '想', '我'},
 {'?*x', '?*y', '想要', '我'},
 {'?*x', '?*y', '喜欢'},
 {'?*x', '?*y', '讨厌'},
 {'?*xAI', '?*y', 'xAI'},
 {'?*x', '?*y', '机器人'},
 {'?*x', '?*y', '对不起'},
 {'?*x', '?*y', '我', '记得'},
 {'?*x', '?*y', '如果'},
 {'?*x', '?*y', '?*z', '我', '梦见'},
 {'?*x', '?*y', '妈妈'},
 {'?*x', '?*y', '爸爸'},
 {'?*x', '?*y', '愿意', '我'},
 {'?*x', '?*y', '因为', '很', '我', '难过', '，'},
 {'?*x', '?*y', '难过'},
 {'?*x', '?*y', '像', '就'},
 {'?*x', '?*y', '?*z', '和', '都'},
 {'?*x', '?*y', '?*z', '一样', '和'},
 {'?*x', '?*y', '我', '是'},
 {'?*x', '?*y', '吗', '我', '是'},
 {'?*x', '?*y', '你', '吗', '是'},
 {'?*x', '?*y', '你', '是'},
 {'?*x', '?*y', '因为'},
 {'?*x', '?*y', '不能', '我'},
 {'?*x', '?*y', '我', '觉得'},
 {'?*x', '?*y', '?*z', '你', '我'},
 {'?*x', '?*y', '不', '为什么', '你'},
 {'?*x', '?*y', '好', '的'},
 {'?*x', '?*y', '嗯'},
 {'?*x', '?*y', '不嘛

In [315]:
e = list(map(lambda x, y: x & y, v1, d))
e

[set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {'我'},
 {'我'},
 set(),
 set(),
 set(),
 set(),
 set(),
 {'我'},
 set(),
 {'我'},
 set(),
 set(),
 {'我'},
 {'我'},
 set(),
 set(),
 set(),
 set(),
 {'我', '是'},
 {'吗', '我', '是'},
 {'你', '吗', '是'},
 {'你', '是'},
 set(),
 {'我'},
 {'我', '觉得'},
 {'你', '我'},
 {'你'},
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 set(),
 {'吗', '是'},
 set()]

In [316]:
e1 = list(map(list, e))
e1

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 ['我'],
 ['我'],
 [],
 [],
 [],
 [],
 [],
 ['我'],
 [],
 ['我'],
 [],
 [],
 ['我'],
 ['我'],
 [],
 [],
 [],
 [],
 ['是', '我'],
 ['是', '我', '吗'],
 ['是', '吗', '你'],
 ['是', '你'],
 [],
 ['我'],
 ['我', '觉得'],
 ['我', '你'],
 ['你'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['是', '吗'],
 []]

In [317]:
def get_max(x):
    return len(x[0]) if x else -1

In [311]:
e3 = max(e1, key=get_max)
e3
set(e3)

{'是'}

In [299]:
last = e.index(set(e3))
last

10

In [259]:
answers = []
for k, v in rule_responses.items():
    if set(new_split(k)) == last:
        answers = v

In [260]:
answers

['你或许现在就能?y', '如果你能?y,会怎样呢？']

In [ ]:
n = list()

In [217]:
list(map(pat_match_with_seg, d, v))

[[('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?xAI', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪']), ('?z', ['不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这样'])],
 [('?x', ['天', '哪', '我', '不能', '这

In [218]:
a = list(filter(lambda x: len(x) > 1, list(map(pat_match_with_seg, d, v))))
a

[[('?x', ['天', '哪']), ('?z', ['不能', '这样'])],
 [('?x', ['天', '哪']), ('?y', ['这样'])],
 [('?x', ['天', '哪']), ('?y', ['不能', '这样'])]]

In [215]:
[pat_to_dict(a) for a in a]

[{'?x': '天 哪', '?z': '不能 这样'},
 {'?x': '天 哪', '?y': '这样'},
 {'?x': '天 哪', '?y': '不能 这样'}]

In [179]:
pat_match_with_seg(new_split('?*x可能?*y'), new_split('我可能厉害'))

[('?x', ['我']), ('?y', ['厉害'])]

In [205]:
new_split('?*X AI ?*y')

['?*X', 'AI', '?*y']

In [207]:
list(jieba.cut('?*xAI?*y'))

['?', '*', 'xAI', '?', '*', 'y']

In [328]:
def get_response_star_new_v2(saying, response_rules=rule_responses):
    patterns = response_rules.keys()
    result = []
    last = []
    saying_list = [saying] * len(patterns)
    set_saying_list = list(map(set, list(map(new_split, saying_list))))
    set_pattern_list = list(map(set, list(map(new_split, list(patterns)))))
    union_best_pattern = list(map(lambda x, y: x & y, set_saying_list, set_pattern_list))

    best_pattern = set_pattern_list[union_best_pattern.index(max(union_best_pattern))]

    for k, v in response_rules.items():
        if set(new_split(k)) == best_pattern:
            temp = pat_match_with_seg(new_split(k), new_split(saying))
            result = substitute(new_split(random.choice(v)), pat_to_dict(temp))

    if not result:
        return random.choice(response_rules['?*x'])
    for each in result:
        last.append(''.join(each.split()))
    return ''.join(last)

In [333]:
def get_response_star_new_v3(saying, response_rules=rule_responses):
    """Make saying to match very single one in response_rules"""
    patterns = response_rules.keys()
    result = []
    last = []
    saying_list = [saying] * len(patterns)
    set_saying_list = list(map(set, list(map(new_split, saying_list))))
    set_pattern_list = list(map(set, list(map(new_split, list(patterns)))))
    union_best_pattern = list(map(lambda x, y: x & y, set_saying_list, set_pattern_list))
    # 此处union_best_pattern中的中文两个词若被jieba分词成一个词，如“讨厌”, 则会判定成一个词，会出现和其他
    # 单个匹配的高频词如“我”等雷同，导致匹配被该种模式截获
    temp_union_1 = list(map(list, union_best_pattern))

    def get_max(x):
        return len(x[0]) if x else -1
    temp_union_2 = set(max(temp_union_1, key=get_max))

    best_pattern = set_pattern_list[union_best_pattern.index(temp_union_2)]

    for k, v in response_rules.items():
        if set(new_split(k)) == best_pattern:
            temp = pat_match_with_seg(new_split(k), new_split(saying))
            result = substitute(new_split(random.choice(v)), pat_to_dict(temp))
            
    if not result:
        return random.choice(response_rules['?*x'])
    for each in result:
        last.append(''.join(each.split()))
    return ''.join(last)

In [301]:
get_response_star_new_v2('天哪我不能这样')

'如果你能这样,会怎样呢？'

In [336]:
get_response_star_new_v2('难道我不能飞')

'如果你能飞,会怎样呢？'

In [338]:
get_response_star_new_v2('大哥你好啊')

'请告诉我你的问题'

In [340]:
get_response_star_new_v2('对不起我想过去')

'你觉得过去有什么意义呢？'

In [342]:
get_response_star_new_v3('我真的喜欢这里')

'你想要这里吗？'

In [343]:
get_response_star_new_v2('昨晚我在梦里梦见他们了')

'你以前梦见过他们了吗'

In [344]:
get_response_star_new_v2('你猜我愿意留下来么')

'你可以解释一下，为什么想留下来么'

In [345]:
get_response_star_new_v2('说真的我很难过，因为下雨了')

'我听到你这么说，也很难过'

In [346]:
get_response_star_new_v2('这样或许更好？')

'你看起来不太确定'

In [351]:
get_response_star_new_v2('那么他们是外星人吗')

'你觉得他们可能不是外星人？'

In [330]:
get_response_star_new_v2('你觉得我是神吗')

'如果你是神，那一位着什么?'

In [334]:
get_response_star_new_v3('我讨厌他')

'讨厌他的哪里？'

## 依然存在不完美匹配，优化代码有种按下葫芦浮起瓢的感觉 =.=